#### Получаем контекст последовательности с помощью pyfaidx
(не учитывая экзон-интронную организацию)

In [1]:
from pyfaidx import Fasta

In [2]:
# поменять на свой
fasta_file_path = 'data/GRCh38.primary_assembly.genome.fa.gz'

In [3]:
fasta = Fasta(fasta_file_path)

In [4]:
chromosome = 'chr18'
position_of_interest = 47245

In [11]:
sequence_of_interest = fasta[chromosome][position_of_interest - 4:position_of_interest + 11]  
# границы выбраны исходя из таблицы tsv просто для проверки

In [12]:
print(sequence_of_interest)

ATTAGTCAAAACCGC


In [14]:
for i, nucleotide in enumerate(sequence_of_interest, 47242):
    print(f"{i} - {nucleotide}")

47242 - A
47243 - T
47244 - T
47245 - A
47246 - G
47247 - T
47248 - C
47249 - A
47250 - A
47251 - A
47252 - A
47253 - C
47254 - C
47255 - G
47256 - C


![nucls](something/nucleotide_positions.png)

#### Пересечение интервалов интересующих позиций

In [3]:
import pandas as pd
from os.path import join

In [29]:
DATA_PATH = "data"
data = pd.read_table(join(DATA_PATH, "deduplicated.tsv"))
data.head()

,Chr,Position,rsID,Ref,Alt,Consequence,Canonical,Gene_symbol,Domains,LoF,LoF_flag,LoF_filter
0,chr18,46050,rs744323,C,T,"['ENST00000308911: downstream_gene_variant', '...","ENST00000308911: YES, ENST00000572530: YES, EN...","TUBB8B, LOC105371950",NaN,NaN,NaN,NaN
1,chr18,47239,.,TTTA,T,"['ENST00000308911: 3_prime_UTR_variant', 'ENST...","ENST00000308911: YES, ENST00000572530: YES, EN...","TUBB8B, LOC105371950",NaN,NaN,NaN,NaN
2,chr18,47242,rs893233952,A,G,"['ENST00000308911: 3_prime_UTR_variant', 'ENST...","ENST00000308911: YES, ENST00000572530: YES, EN...","TUBB8B, LOC105371950",NaN,NaN,NaN,NaN
3,chr18,47243,.,T,C,"['ENST00000308911: 3_prime_UTR_variant', 'ENST...","ENST00000308911: YES, ENST00000572530: YES, EN...","TUBB8B, LOC105371950",NaN,NaN,NaN,NaN
4,chr18,47245,.,A,T,"['ENST00000308911: 3_prime_UTR_variant', 'ENST...","ENST00000308911: YES, ENST00000572530: YES, EN...","TUBB8B, LOC105371950",NaN,NaN,NaN,NaN


In [7]:
data.shape

(339, 12)

Из распарсенной таблицы отобрали только строки, содержащие непустые значения в одном из лоф-полей (LoF/LoF_flag/LoF_filter)  

In [30]:
filtered_data = data[(~data['LoF'].isna()) | (~data['LoF_flag'].isna()) | (~data['LoF_filter'].isna())]

In [21]:
filtered_data.shape

(10, 12)

Перевели в vcf

In [32]:
vcf_df = filtered_data[['Chr', 'Position', 'rsID', 'Ref', 'Alt']] 
vcf_df.loc[:, 'QUAL'] = "."
vcf_df.loc[:, 'FILTER'] = "."
vcf_df.loc[:, 'INFO'] = "."

header = """##fileformat=VCFv4.2
#CHROM POS ID REF ALT QUAL FILTER INFO
"""

output_VCF = "data/myfile.vcf"
with open(output_VCF, 'w') as vcf:
    vcf.write(header)

vcf_df.to_csv(output_VCF, sep="\t", mode='a', index=False, header=False)

/tmp/ipykernel_4995/4217350662.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vcf_df.loc[:, 'QUAL'] = "."
/tmp/ipykernel_4995/4217350662.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vcf_df.loc[:, 'FILTER'] = "."
/tmp/ipykernel_4995/4217350662.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

Сделала  
! bedtools intersect -a gencode.v45.annotation.gtf.gz -b myfile.vcf > intersected_annotations.gtf

Получился файл intersected_annotations.gtf, но я не совсем понимаю, что с ним делать дальше :/